In [ ]:
# IMPORTS
import random as rnd
import numpy as np

In [ ]:
def debug_print(lista, titulo):
    print(titulo)
    for i in lista:
        print(i.fitness)

In [ ]:
"""
CLASE INDIVIDUO

 Atributos:
 - x1: gen 1, primera variable de la función
 - x2: gen 2, segunda variable de la función
 - fitness: valor de la función evaluada en x1 y x2
 """

class individuo:
    def __init__(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.fitness = 0
    
    def set_fitness(self, f):
        self.fitness = f
        #print(self.fitness)
        
    def debug(self):
        s = "x1: "+ str(self.x1) + "\n" + "x2: "+ str(self.x2) + "\n" + "fitness: "+ str(self.fitness)
        return s
        

In [ ]:
"""
POOL_INICIAL
Construye el pool inicial

 Parámetros:
 - n_individuos: cantidad de individuos en la población inicial
 - limsx1: límite superior del gen 1
 - limix1: límite inferior del gen 1
 - limsx2: límite superior del gen 2
 - limix2: límite inferior del gen 2
 - restr: restricción de la relación entre x1 y x2
 Salida:
 - population: lista de individuos de la población inicial
 """
def pool_inicial(n_individuos, limsx1, limix1, limsx2, limix2, restr):
    population = []
    for i in range(n_individuos):
        x1 = round(rnd.random()*(limsx1 - limix1)+limix1, 3)
        #el límite superior depende de la restricción
        limrx2 = restr(x1)
        limsx2 = min(limsx2,limrx2) 
        x2 = round(rnd.random()*(limsx2 - limix2)+limix2, 3)
        # creación del objeto individuo
        ind = individuo(x1,x2) 
        # inclusión del individuo en la población
        population.append(ind)
    return population

In [ ]:
"""
CHECK_MUTATION
Verifica que los genes cumplan con las restricciones

 Parámetros:
 - x1: primer gen del individuo
 - x2: segundo gen del individuo
 - limsx1: límite superior del gen 1
 - limix1: límite inferior del gen 1
 - limsx2: límite superior del gen 2
 - limix2: límite inferior del gen 2
 - restr: restricción de la relación entre x1 y x2 
 Salida: 
 - check_x1: booleano que indica si x1 cumple con las restricciones
 - check_x2: booleano que indica si x2 cumple con las restricciones
"""
def check_mutation(x1, x2, limsx1, limix1, limsx2, limix2, restr):
    check_x1 = False
    check_x2 = False
    limrx2 = restr(x1)
    limsx2 = min(limsx2,limrx2)
    if(x1<limsx1 and x1>limix1):
        check_x1 = True
    if(check_x1):
        limrx2 = restr(x1)
        limsx2 = min(limsx2,limrx2)
        if(x2<limsx2 and x2>limix2):
            check_x2 = True    
    return check_x1, check_x2

In [ ]:
"""
FITNESS_CALCULATION
Calcula el fitness de cada individuo en la población

 Parámetros:
 - population: población
 - fun: función de fitness
"""
def fitness_calculation(population, fun):
    for i in population:
        #print("indv x1: ", i.x1, "indv x2: ", i.x2)
        i.set_fitness(fun(i.x1, i.x2))

In [ ]:
"""
SELECTION
Selecciona la mitad de la población con el mejor fitness

 Parámetros: 
 - population: población
 Salida: 
 - new_population: población seleccionada
 """
def selection(population):
    # se ordenan los individuos según su fitness (de mayor a menor)
    pop_sorted = sorted(population, key=lambda x: x.fitness, reverse=True)
    # se guarda únicamente la mitad con los mejores fitness
    new_population = pop_sorted[:round(len(pop_sorted)/2)]
    return new_population

In [ ]:
"""
CROSS
Realiza el cruce de individuos para obtener la próxima generación

 Parámetros:
 - population: población
 Salida: 
 - new_population: nueva población de individuos luego del cruce
"""
def cross(population):
    new_population = []
    # se realiza la combinación de todos los individuos
    for i in population:
        for j in population:
            if(i != j):
                x1_heredada = i.x1
                x2_heredada = j.x2
                hijo = individuo(x1_heredada, x2_heredada)
                new_population.append(hijo)
    # Se mantiene el mismo número de individuos en la siguiente
    # generación. Como los individuos están ordenados según el 
    # fitness, se tendrán más hijos de los cruces de padres con
    # mejor fitness
    return new_population[:len(population)*2]

In [ ]:
"""
MUTATION
Realiza la mutación de individuos en la población

 Parámetros:
 - population: población cruzada
 - limsx1: límite superior del gen 1
 - limix1: límite inferior del gen 1
 - limsx2: límite superior del gen 2
 - limix2: límite inferior del gen 2
 - restr: restricción de la relación entre x1 y x2
 Salida:
 - new_population: población mutada

"""
def mutation(population, limsx1, limix1, limsx2, limix2, restr):
    new_population = []
    for i in population:
        c_x1 = False
        c_x2 = False
        n = 0
        # mientras no se cumpla con las restricciones generar una nueva
        # mutación
        while(c_x1 == False or c_x2 == False):
            # se selecciona el gen a mutar de forma aleatoria
            gen_mutar = rnd.randint(0,1)
            # los primeros 10 intentos se realizan con un random en
            # distribución gaussiana para que no se aleje tanto del 
            # gen inicial
            if(n<10):
                if(gen_mutar == 0):
                    x1 = round(i.x1 + rnd.gauss(0, 0.1), 3)
                    x2 = i.x2
                else:
                    x1 = i.x1
                    x2 = round(i.x2 + rnd.gauss(0, 0.1), 3)
            # si no se logra obtener una mutación valida en los primeros
            # 10 intentos se elige un random uniforme para agregar
            # variabilidad
            else:
                if(gen_mutar == 0):
                    x1 = round(rnd.random()*(limsx1 - limix1)+limix1, 3)
                    x2 = i.x2
                else:
                    x1 = i.x1
                    x2 = round(rnd.random()*(limsx2 - limix2)+limix2, 3)
            
            n += 1
            # se verifica la mutación
            c = check_mutation(x1, x2, limsx1, limix1, limsx2, limix2, restr)
            c_x1 = c[0]
            c_x2 = c[1]
        # Si la mutación cumple con las restricciones se agrega el nuevo
        # individuo a la población mutada
        new_population.append(individuo(x1,x2))
    return new_population

In [ ]:
"""
OPTIMO
Obtiene el individuo con el mejor fitness luego de cierto número de 
generaciones. 

 Parámetros:
 - fitness: función de fitness
 - n_individuos: cantidad de individuos en la población inicial
 - limsx1: límite superior del gen 1
 - limix1: límite inferior del gen 1
 - limsx2: límite superior del gen 2
 - limix2: límite inferior del gen 2
 - restr: restricción de la relación entre x1 y x2
 - nmax: número de generaciones 
 Salida:
 - population[0]: individuo con el mejor fitness
"""
def Optimo(fitness, n_individuos, limsx1, limix1, limsx2, limix2, restr, nmax):
    varianza = 1
    n = 0
    # Población inicial
    population = pool_inicial(n_individuos, limsx1, limix1, limsx2, limix2, restr)
    fitness_calculation(population, fitness)
    while(n < nmax):
        #print de debug
        print("\n----> iteración: ", n)
        print("cant ind", len(population))
        print("first item: ", population[len(population)-1].x1, " ", population[len(population)-1].x2)
        
        # SELECCIÓN
        population = selection(population)
        #debug_print(population, "after selection")
        
        # CRUCE
        population = cross(population)
        #debug_print(population, "after cross")
        
        # MUTACIÓN
        population = mutation(population, limsx1, limix1, limsx2, limix2, restr)
        
        # CÁLCULO DE FITNESS
        fitness_calculation(population, fitness)
        #debug_print(population, "after mutation")
        
        n += 1
        print("fitness order first item: ", population[0].fitness)
    return population[0]

In [ ]:
fitness = lambda x1,x2: -(x1)**2 - (x2)**2 + 5
restr = lambda x1: x1
pos = np.linspace(-5,5,100)

o = Optimo(fitness, 30, 5, -5, 5, -5, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)

# Task 1
## Función: 
$$f(x_1,x_2) = 15x_1 + 30x_2 + 4x_1x_2 - 2x_1^2 - 4x_2^3$$

## Restricciones: 
$$x_1 + 2x_2 \leq 30$$
$$x_1 \geq 0, x_2 \geq 0$$

In [ ]:
fitness = lambda x1,x2: (15*x1)+(30*x2)+(4*x1*x2)-(2*(x1**2))-(4*(x2**2))
restr = lambda x1: 15 - (x1/2)
o = Optimo(fitness, 15, 40, 0, 40, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)

# Task 2
## Función:
$$z = 3x_1 + 5x_2$$
## Restricciones:
$$x_1 \leq 4$$
$$2x_2 \leq 12$$
$$3x_1 + 2x_2 \leq 18$$
$$x_1 >= 0, x_2 \geq 0$$

In [ ]:
fitness = lambda x1,x2: (3*x1) + (5*x2)
restr = lambda x1: (18.0/2.0) - (3.0/2.0)*x1
o = Optimo(fitness, 15, 4, 0, 6, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)

# Task 3
## Función:
$$f(x_1,x_2) = 5x_1 - x_1^2 + 8x_2 - 2x_2^2$$
## Restricciones:
$$3x_1 + 2x_2 \leq 6$$
$$x_1 \geq 0, x_2 \geq 0$$

In [ ]:
fitness = lambda x1,x2: (5*x1) - (x1**2) + (8*x2) - (2*(x2**2))
restr = lambda x1: (6.0/2.0) - (3.0/2.0)*x1
o = Optimo(fitness, 15, 2, 0, 3, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)

# Task 4
# Problema del cartero viajante
El problema consiste optimizar la trayectoria de un viajero que debe visitar todas las ciudades disponibles una sola vez y cada una de las ciudades se encuentra a una distancia específica de las otras.

## Alelos:
Los alelos en este problema son numeros de 1 a 8 que representan las ciudades.

## Genoma:
El genoma es la combinación de cuidades y el orden de los alelos representa la trayectoria a seguir. En este caso se representa por medio de un arreglo como el siguiente:
$$[1, 4, 8, 7, 6, 5, 3, 2]$$

## Fenotipo:
En el problema se evaluaron 14 fenotipos distintos (14 individuos en la población). En una ejecución de algoritmos genéticos el fenotipo es generado aleatoriamente. En este caso son arreglos de numeros que indican el recorrido completo y ordenado.

## Fitness:
Es la función o problema que se desea optimizar. En este caso es minimizar la distancia total que se recorre para visitar cada una de las ciudades. A cada genoma se le calcula la distancia recorrida y la distancia total es el valor de fitness.

## Criterios de selección:
Para este problema se usó un criterio de selección de tipo ruleta, en este tipo de selección cada uno de los genomas tiene una probabilidad distinta de ser seleccionado con base en el valor de fitness que posea. Para este problema entre menor distncia mayor la probabilidad de ser seleccionado.

## Criterios de cruce:
El tipo de cruce fue de tipo simple, este tipo de cruce se realiza dividiendo el genoma en un punto definido, como la mitad del genoma. El nuevo individuo obtiene los alelos del padre hasta el punto seleccionado y la otra corresponde a los alelos de la madre desde el punto hasta el final de los alelos.

## Criterios de mutación:
La tasa de mutación utilizado para este problema fue de 0.02. Las mutaciones normalmente se realizan seleccionando un gen de forma aleatoria y se genera un número aleatorio con distribución gausiana.

## Referencia:
https://www.redalyc.org/pdf/849/84912053047.pdf